In [1]:
import openrouteservice as ors
import folium
from folium.plugins import PolyLineTextPath
from shapely.geometry import Point, Polygon
from collections import Counter



In [ ]:
client = ors.Client(key='YOUR_ORS_API_KEY')  # Replace with your actual OpenRouteService API key

In [ ]:
## '''Running Route Generator!'''
'''By Stylianos Regas'''

from shapely.geometry import shape, Point, LineString
import folium
import random


'''OutnBackGenerator Function'''
'''creates a route that goes out to a random point, then back on that same route'''
def OutnBackGen(focus_point,use_miles,quantity,pace):

    #creates isochrone based off of time
    if(use_miles == False):
        
        quantity = ((quantity/pace)*1609.344)/2
        
        #Gets isochrone based off input time
        isochrone = client.isochrones(
            locations=[focus_point],
            profile='foot-walking',
            range_type='distance',
            range=[quantity],  #has range based off of time 
            
        )
        
        
        
    #creates isochrone based off of mileage
    elif(use_miles == True):
        
        quantity = (quantity*1609.344)/2
       
        #Get isochrone based off input miles
        isochrone = client.isochrones(
            locations=[focus_point],
            profile='foot-walking',
            range_type='distance',
            range=[(quantity)],  #has range based off of miles
            
        )
        
    # finds boundary points of the isochrone
    iso_polygon = shape(isochrone['features'][0]['geometry'])
    boundary_coords = list(iso_polygon.exterior.coords)

    # picks a random coord along the boundary
    center = Point(focus_point)
    random_point = boundary_coords[random.randint(0, len(boundary_coords)-1)]

    # creates the route from the starting point to the boundary point selected
    route = client.directions(
        coordinates=[focus_point, list(random_point), focus_point],
        profile='foot-walking',
        format='geojson',
      
    )

    return route,random_point


'''loopGenerator Function'''
'''creates a loop by getting several points on the isochrone and plotting the route with that'''
def LoopGen(focus_point,use_miles,quantity,pace):

    offset = 0
    
    if(use_miles == False):
        
        quantity = quantity/pace
        offset = quantity
        quantity = (quantity*1609.344)/6
        
        #Gets isochrone based off input time
        isochrone = client.isochrones(
            locations=[focus_point],
            profile='foot-walking',
            range_type='distance',
            range=[quantity],  #has range based off of time 
            
        )
        
        
        
    #creates isochrone based off of mileage
    elif(use_miles == True):
    
        offset = quantity
        quantity = (quantity*1609.344)/4.5
       
        #Get isochrone based off input miles
        isochrone = client.isochrones(
            locations=[focus_point],
            profile='foot-walking',
            range_type='distance',
            range=[(quantity)],  #has range based off of miles
            
        )

    #makes the offset either positive or negative to add randomness to the generator
    offset = offset*random.choice([-1, 1])
    
    # finds boundary points of the isochrone
    iso_polygon = shape(isochrone['features'][0]['geometry'])
    boundary_coords = list(iso_polygon.exterior.coords)
    
    
    # picks a random coord along the boundary
    center = Point(focus_point)
    random_index = random.randint(0, len(boundary_coords)-1)
    
    
    #Adds extra points on the map to have a more looping route
    x = random_index+int(offset)
    y = random_index+int(offset)*2
    z = random_index+int(offset)*3
    
    #length of the total number of coordinates on the isochrone
    length = len(boundary_coords)
    
    #checks to see if all the values are in the boundary coords, if not, they are divided to fit in the coords
    if(x>= length):
        x = (length-1) % x
    
    elif(x<= -1*length):
        x = (length-1) % x
    
    if(y>= length):
        y = (length-1) % y
            
    elif(y<= -1*length):
        y = (length-1) % y
    
    if(z>= len(boundary_coords)):
        z = (length-1) % z
       
    elif(z<= -1*length):
        z = (length-1) % z 


    
    #initializes all the points
    random_point = boundary_coords[random_index]
    random_point1 = boundary_coords[x]
    random_point2 = boundary_coords[y]
    random_point3 = boundary_coords[z]
  
    
    # creates the route from the starting point to the boundary point selected
    route = client.directions(
        coordinates=[focus_point, list(random_point3), list(random_point2), list(random_point1),list(random_point),focus_point],
        profile='foot-walking',
        format='geojson',
        instructions = True
    )
    return route,random_point,random_point1,random_point2,random_point3
  
    

#focus_point initialization


offset = 0 #determines how far across the isochrone the second point will be based off of miles or time

#title
print("="*50,"Running Route Generator!","="*50)
print("-"*49,"Created by Styilanos Regas","-"*49)
print("*"*19,"NOTE: depending on location and surrounding roads, routes may vary in desired out come","*"*19)
print("First, we need to get coordinates to start with!")


# 1. Gets coordinates either manually or from address
while True:
    focus_point = 0
    
    choice = input('Enter "address" to base location off of address. Enter "coordinates" to base location off of coordinates')

    #uses geocoding to recieve coordinates from address
    if(choice == 'address'):
        
        address = input('Enter in the address you want to start at. Example address: "123 Smith St, Smithville, AB, 12345"')
        geocode = client.pelias_search(text=address)
        focus_point = geocode['features'][0]['geometry']['coordinates']
  
    
    #gets coordinates manually
    elif(choice == 'coordinates'):
        lat = input('enter the latitude coordinates')
        lon = input('enter the longitude coordinates')
        focus_point = [lat, lon]
        print(focus_point)
        
    else:
        print("invald choice!")

    if(focus_point != 0):
        break
while True:
    
    #2. asks if they want one of the two options for the route. Out and back or a loop
    print("1. Generate a route that goes out to a certain point, then back right following the same path")
    print("2, Generate a route that generates several points to create a loop around the starting point")

    #1 is out and back, 2 is loop
    type = input("enter the number of which path you want to create")
    if(type == "1"):
        type = 1
        break
    elif(type == "2"):
        type = 2
        break
    
    else:
        print("Invalid response")
            

while True:
    
    #Asks user if they want to run based off of time or mileage to determine what kind of isochrone to make
    dependent = input('enter "miles" to run off mileage or "time" to run off time')

    #creates isochrone based off of time
    if(dependent == "time" ):
        time = (float(input("enter the amount of time in minutes you want to run for")))
        pace = (float(input("enter the pace you want to run each mile at")))

        if(time > 600):
            print("Error! Exceeds time limit!")

        else:
            counter = 1
            
            if(type == 1):
               route,random_point = OutnBackGen(focus_point,False,time,pace) 
            else:
                route,random_point,random_point1,random_point2,random_point3 = LoopGen(focus_point,False,time,pace)
    
            while True:
                counter = counter + 1
                
                
                #checks to see if time if is close enough to the desired amount
                total_duration = sum(
                    segment['duration']
                    for segment in route['features'][0]['properties']['segments']
                )
                
                total_minutes = total_duration/60

    
                if(total_minutes>= (time*.85) and total_minutes <= (time*1.15)):
                    break

                elif(total_minutes > time*.85):
                    if(total_minutes > time*.75):
                        if(type == 1):
                            route,random_point = OutnBackGen(focus_point,False,time*.8,pace)
                        else:
                            route,random_point,random_point1,random_point2,random_point3 = LoopGen(focus_point,False,time*.8,pace)
                        
                    else:
                        if(type == 1):
                            route,random_point = OutnBackGen(focus_point,False,time*.9,pace) 
                        else:
                            route,random_point,random_point1,random_point2,random_point3 = LoopGen(focus_point,False,time*.9,pace)
                    

                elif(total_minutes < time*1.15):
                    if(total_minutes < time*1.25):
                        if(type == 1):
                            route,random_point = OutnBackGen(focus_point,False,time*1.2,pace)
                        else:
                            route,random_point,random_point1,random_point2,random_point3 = LoopGen(focus_point,False,time*1.2,pace)
                    else:
                        if(type == 1):
                            route,random_point = OutnBackGen(focus_point,False,time*1.1,pace) 
                        else:
                            route,random_point,random_point1,random_point2,random_point3 = LoopGen(focus_point,False,time*1.1,pace)
                    
                if(counter >=5 ):
                    break

                
            break
        
    #creates isochrone based off of mileage
    elif(dependent == "miles"):
        miles = (float(input("enter the amount of mileage you want to run for")))

        if(miles > 100):
            print("Error! Exceeds miles limit!")

        else:
            counter = 1
            if(type == 1):
                route,random_point = OutnBackGen(focus_point,True,miles,0) 
            else:
                route,random_point,random_point1,random_point2,random_point3 = LoopGen(focus_point,True,miles,0)
                
            while True:
                
                #determines if mileage is close enough to desired amount, reruns the loop if not
                distance = sum(
                    segment['distance']
                    for segment in route['features'][0]['properties']['segments']
                )
                total_distance = distance / 1609.34

                if(total_distance <= miles*1.15 and total_distance >= miles*.85):
                    break
                
                elif(total_distance> (miles*1.15)):
                    
                    if(total_distance > (miles*1.25)):
                        if(type == 1):
                            route,random_point = OutnBackGen(focus_point,True,miles*.8,0) 
                        else:
                            route,random_point,random_point1,random_point2,random_point3 = LoopGen(focus_point,True,miles*.8,0)
                        
                    else:
                        if(type == 1):
                            route,random_point = OutnBackGen(focus_point,True,miles*.9,0)
                        else:
                            route,random_point,random_point1,random_point2,random_point3 = LoopGen(focus_point,True,miles*.9,0)   

                elif(total_distance < (miles*.85)):
                    
                    if(total_distance < (miles*.75)):
                        if(type == 1):
                            route,random_point = OutnBackGen(focus_point,True,miles*1.2,0)
                        else:
                            route,random_point,random_point1,random_point2,random_point3 = LoopGen(focus_point,True,miles*1.2,0)
                    else:
                        if(type == 1):
                            route,random_point = OutnBackGen(focus_point,True,miles*1.1,0)
                        else:
                            route,random_point,random_point1,random_point2,random_point3 = LoopGen(focus_point,True,miles*1.1,0)
                        
                counter = counter + 1
                if(counter > 5):
                    break
                
            break
    else:
        print("that is not an option!")

#displays the map
map_route = folium.Map(location=[focus_point[1], focus_point[0]], zoom_start=13,tiles='cartodbpositron')


#displays route
folium.GeoJson(route, name="Route").add_to(map_route)

folium.Marker(location=[focus_point[1], focus_point[0]], tooltip="Start",icon=folium.Icon(color='green',icon = 'home')).add_to(map_route)

if(type == 1):
    
    folium.Marker(location=[random_point[1], random_point[0]], tooltip="turn around",icon=folium.Icon(color='blue',icon = 'flag')).add_to(map_route)
    
else:
    folium.Marker(location=[random_point[1], random_point[0]], tooltip="checkpoint 4",icon=folium.Icon(color='blue',icon = 'flag')).add_to(map_route)
    folium.Marker(location=[random_point1[1], random_point1[0]], tooltip="checkpoint 3",icon=folium.Icon(color='blue',icon = 'flag')).add_to(map_route)
    folium.Marker(location=[random_point2[1], random_point2[0]], tooltip="checkpoint 2",icon=folium.Icon(color='blue',icon = 'flag')).add_to(map_route)
    folium.Marker(location=[random_point3[1], random_point3[0]], tooltip="checkpoint 1",icon=folium.Icon(color='blue',icon = 'flag')).add_to(map_route)




#changes route color
routeLine = folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']], color="grey").add_to(map_route)

#adds arrows to map
PolyLineTextPath(
    routeLine,
    ">",  # Arrow symbol
    repeat=True,
    offset=6.1,
    attributes={"fill": "grey", "font-size": "18"}
).add_to(map_route)

#prints mileage total
distance = sum(
    segment['distance']
    for segment in route['features'][0]['properties']['segments']
)
total_distance = distance / 1609.34
print(f"total distance: {total_distance:.2f} miles" )

#prints time total
if(dependent == "miles"):
    total_duration = sum(
        segment['duration']
        for segment in route['features'][0]['properties']['segments']
    )
    total_minutes = total_duration/60
    print(f"Total route duration(12min/mile): {total_minutes:.2f} minutes")

elif(dependent == "time"):
    
    total_minutes = total_distance*pace
    print(f"Total route duration: {total_minutes:.2f} minutes")
    
    





#displays route directions
segments = route['features'][0]['properties']['segments']

print("\n====Instructions====")
for seg_idx, segment in enumerate(segments):
    print(f"\n--- Segment {seg_idx + 1} ---")
    for i, step in enumerate(segment['steps']):
        instruction = step['instruction']
        dist = round(step['distance'])
        dur = round(step['duration'] / 60, 1)
        print(f"{i+1}. {instruction} ({dist}m, {dur} min)")


map_route




